# Table of Contents
1. [Load the Data](#1.-Load-the-Data)
2. [Some First Eyeballing](#2.-Some-First-Eyeballing)
3. [Data Cleaning](#3.-Data-Cleaning)
4. [Eyeballing After Cleaning](#4.-Eyeballing-After-Cleaning)
5. [Descriptive Statistics](#5.-Descriptive-Statistics)
    - [Average Time Needed](#Average-Time-Needed)
    - [Visualize Demographics](#Visualize-Demographics-(before-and-after-cleaning))
    - [Test For Age Differences](#Test-For-Age-Differences)
    - [Test For Gender Differences](#Test-For-Gender-Differences)


6. [Statistical Analysis (finally!)](#6.-Statistical-Analysis-(finally!)) 
    - [First: Accuracy](#First:-Accuracy)
    - [Second: User Judgements from Survey](#Second:-User-Judgements-from-Survey)

# 1. Load the Data
First, we'll load all the CSV files into pandas DataFrames. We will also do some slight re-coding of the data for easier processing.

In [ ]:
%pip install pandas numpy matplotlib seaborn scipy scikit_posthocs statsmodels rpy2

In [ ]:
# Step A: Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# Step B: Load the data
demographics = pd.read_csv('https://raw.githubusercontent.com/andreArtelt/IJCAI24-CF_Tut/main/CESORP_data_final/demographics.csv')
reaction_times_instructions = pd.read_csv('https://raw.githubusercontent.com/andreArtelt/IJCAI24-CF_Tut/main/CESORP_data_final/reactionTimesInstructions.csv')
reaction_times_task = pd.read_csv('https://raw.githubusercontent.com/andreArtelt/IJCAI24-CF_Tut/main/CESORP_data_final/reactionTimesTask.csv')
survey_data = pd.read_csv('https://raw.githubusercontent.com/andreArtelt/IJCAI24-CF_Tut/main/CESORP_data_final/surveyData.csv')
task_performance = pd.read_csv('https://raw.githubusercontent.com/andreArtelt/IJCAI24-CF_Tut/main/CESORP_data_final/taskPerformance.csv')

# Step C: task performance, re-code Y to 1 and N to 0, add accuracy
task_performance['userPred'] = task_performance['userPred'].map({'Y': 1, 'N': 0})
task_performance['truePred'] = task_performance['truePred'].map({'Y': 1, 'N': 0})
task_performance['accuracy'] = (task_performance['userPred'] == task_performance['truePred']).astype(int)

# Step D: demographics, clean up superflous whitespace in categorical columns
demographics['gender'] = demographics['gender'].str.replace(r'\s{2,}', ' ', regex=True).str.strip()
demographics['age'] = demographics['age'].astype(str).str.replace(r'\s{2,}', ' ', regex=True).str.strip()

# Step E: Define a consistent color palette for consistent plotting
color_palette = {
    's-cfe': '#1f77b4',  # blue
    'o-cfe': '#ff7f0e',  # orange
    's-con': '#2ca02c',  # green
    'o-con': '#d62728'   # red
}

# Convert the palette to a list of colors
colors = [color_palette[key] for key in color_palette]

# Step F: Define a consistent color palette for consistent plotting
color_palette_interaction = {
    'cfe': '#1f77b4',  # blue
    'con': '#ff7f0e',  # orange
}

# Convert the palette to a list of colors
colors = [color_palette[key] for key in color_palette]

#reaction_times_instructions['totalTimeMins']=reaction_times_instructions['totalTime']/1000/60
#print(reaction_times_instructions.sort_values(by=['totalTime'], ascending=True))

# 2. Some First Eyeballing

In [ ]:
def get_ylim(data, group_column, value_column):
    grouped = data.groupby(group_column)[value_column].agg(['mean', 'sem']).reset_index()
    max_ylim = (grouped['mean'] + 1.5*grouped['sem']).max()
    return 0, max_ylim

def get_material_ylim(data, group_column, material_column, value_column):
    grouped = data.groupby([group_column, material_column])[value_column].agg(['mean', 'sem']).reset_index()
    max_ylim = (grouped['mean'] + 1.2*grouped['sem']).max()
    return 0, max_ylim

# Initial plotting for eye-balling the data
plt.figure(figsize=(8, 4))

# Determine the maximum count for setting the y-limits for demographics
age_counts = demographics.groupby(['age', 'cond']).size().max()
gender_counts = demographics.groupby(['gender', 'cond']).size().max()
max_count_demo = max(age_counts, gender_counts)

# Define all possible categories for age and gender
all_ages = ['18-24y', '25-34y', '35-44y', '45-54y', '55-65y', '65y and older', 'prefer not to say']  # Assuming ages range from 20 to 30
all_genders = ['male', 'female', 'non-binary', 'transgender', 'other', 'prefer not to say']  # Assuming genders include Male, Female, Other

# Convert age and gender columns to category type with all possible categories
demographics['age'] = pd.Categorical(demographics['age'], categories=all_ages)
demographics['gender'] = pd.Categorical(demographics['gender'], categories=all_genders)

# Plot Age distribution
plt.subplot(1, 2, 1)
sns.countplot(data=demographics, x='age', hue='cond',palette=color_palette)
plt.title('Age Distribution by Condition')
plt.xlabel('Age')
plt.ylabel('Count')
plt.xticks(rotation=90)
plt.legend(loc='upper right')
plt.ylim(0, max_count_demo + 1)

# Plot Gender distribution
plt.subplot(1, 2, 2)
sns.countplot(data=demographics, x='gender', hue='cond',palette=color_palette)
plt.title('Gender Distribution by Condition')
plt.xlabel('Gender')
plt.ylabel('Count')
plt.xticks(rotation=90)
plt.legend(loc='upper right')
plt.ylim(0, max_count_demo + 1)
plt.tight_layout()
plt.show()

# Look at Accuracy
# add a column 'orderedTrial' reflecting the temporal order of trials per participant
task_performance = task_performance.copy()
task_performance['orderedTrial'] = task_performance.groupby('userID').cumcount() + 1

plt.figure(figsize=(8, 4))
# Plot Mean Accuracy per user
# Compute mean accuracy and merge with demographics for plotting
mean_accuracy_per_user = task_performance.groupby('userID')['accuracy'].mean().reset_index()
mean_accuracy_per_user = mean_accuracy_per_user.merge(demographics, on='userID')
sns.barplot(data=mean_accuracy_per_user, x='cond', y='accuracy', palette=color_palette, errorbar='se')
plt.title('Mean Accuracy per User by Condition')
plt.xlabel('Condition')
plt.ylabel('Mean Accuracy')
plt.ylim(get_ylim(mean_accuracy_per_user, 'cond', 'accuracy'))
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 4))
# Plot Mean Accuracy over users of each group over time
# (= trials chronologically ordered, to eyeball effects related to the duration of the study, fatigue, etc.)
mean_performance_per_trial = task_performance.groupby(['cond', 'orderedTrial']).agg({'accuracy': 'mean'}).reset_index()
sns.lineplot(data=mean_performance_per_trial, x='orderedTrial', y='accuracy', hue='cond', marker='o', palette=color_palette)
plt.title('Mean Accuracy Over Users of Each Group on Each Trial - Chronologically Ordered')
plt.xlabel('Trial Number')
plt.ylabel('Mean Accuracy')
plt.legend(title='Condition')
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 4))
# Plot Mean Accuracy over users of each group on each material as bar plots
sns.barplot(data=task_performance, x='trialNo', y='accuracy', hue='cond', palette=color_palette, errorbar='se')
plt.title('Mean Accuracy Over Users of Each Group on Each Material')
plt.xlabel('Trial Number')
plt.ylabel('Mean Accuracy')
plt.ylim(get_material_ylim(task_performance, 'cond', 'trialNo', 'accuracy'))
plt.legend(title='Condition')
plt.tight_layout()
plt.show()

# Look at Reaction Times
# add a column 'orderedTrial' reflecting the temporal order of trials per participant
reaction_times_task = reaction_times_task.copy()
reaction_times_task['orderedTrial'] = reaction_times_task.groupby('userID').cumcount() + 1

max_ylim_times_mean=max(get_ylim(reaction_times_task, 'cond', 'responseTime'),get_ylim(reaction_times_task, 'cond', 'explanationTime'))

plt.figure(figsize=(8, 4))
# Plot Reaction Times
plt.subplot(1, 2, 1)
sns.barplot(data=reaction_times_task, x='cond', y='responseTime', palette=color_palette, errorbar='se')
plt.title('Mean Response Time by Condition')
plt.xlabel('Condition')
plt.ylabel('Response Time (ms)')
plt.ylim(max_ylim_times_mean)

plt.subplot(1, 2, 2)
sns.barplot(data=reaction_times_task, x='cond', y='explanationTime', palette=color_palette, errorbar='se')
plt.title('Mean Explanation Time by Condition')
plt.xlabel('Condition')
plt.ylabel('Explanation Time (ms)')
plt.ylim(max_ylim_times_mean)
plt.tight_layout()
plt.show()

# Plot reaction times per user, divided into groups for more visibility
plt.figure(figsize=(8, 12))
groups = reaction_times_task['cond'].unique()
response_time_max = reaction_times_task['responseTime'].max()
explanation_time_max = reaction_times_task['explanationTime'].max()

# Trials Chronologically Ordered, to Eyeball Effects Related to the Duration of the Study, Fatigue, etc.)
for i, group in enumerate(groups, 1):
    plt.subplot(len(groups), 1, i)
    subset = reaction_times_task[reaction_times_task['cond'] == group]
    for user in subset['userID'].unique():
        user_data = subset[subset['userID'] == user]
        plt.plot(user_data['orderedTrial'], user_data['responseTime'], marker='o', linestyle='-', color='blue', alpha=0.6)
        plt.plot(user_data['orderedTrial'], user_data['explanationTime'], marker='x', linestyle='--', color='red', alpha=0.6)
    plt.title(f'Reaction Times for Group over Time: {group}')
    plt.xlabel('Trial Number (Chronological)')
    plt.ylabel('Time (ms)')
    plt.ylim(0, max(response_time_max, explanation_time_max))
    plt.legend(['Response Time', 'Explanation Time'], loc='upper right')
plt.tight_layout()
plt.show()

max_ylim_times=max(get_material_ylim(reaction_times_task, 'cond', 'trialNo', 'explanationTime'),get_material_ylim(reaction_times_task, 'cond', 'trialNo', 'responseTime'))

# Plot Response Time per Material divided by group (to spot Effects Related to Materials)
plt.figure(figsize=(8, 4))
sns.barplot(x='trialNo', y='responseTime', hue='cond', data=reaction_times_task, palette=color_palette, errorbar='se')
plt.title('Mean Response Time per Material Number')
plt.xlabel('Material Number')
plt.ylabel('Response Time (ms)')
plt.ylim(max_ylim_times)
plt.legend(title='Condition')
plt.tight_layout()
plt.show()

# Plot Explanation Time per Material divided by group (to spot Effects Related to Materials)
plt.figure(figsize=(8, 4))
sns.barplot(x='trialNo', y='explanationTime', hue='cond', data=reaction_times_task, palette=color_palette, errorbar='se')
plt.title('Mean Explanation Time per Material Number')
plt.xlabel('Material Number')
plt.ylabel('Explanation Time (ms)')
plt.ylim(max_ylim_times)
plt.legend(title='Condition')
plt.tight_layout()
plt.show()

# 3. Data Cleaning
Next, we'll clean the data based on pre-defined criteria.

#### Outliers relative to population mean
Specifically, we want to remove:

- all speedsters  >3 SDs below the mean​
- all materials  >3 SDs from mean​
- materials >3 SDs from person‘s own mean, to catch attention failures​
- straight-liners, repeatedly giving same answer, even with negative feedback​
- non-varying responses


In [ ]:
# Define a function for removing outliers
def identify_outliers(df, column):
    # define the factor with which std will be multiplied to find admissible range
    removal_factor_population=3
    print("\nMeasure: " + column)
    print("Population mean: " + str(round(df[column].mean(),3)))
    print("Population std: " + str(round(df[column].std(),3)))
    print("Admissible range: " + str(round(df[column].mean()-removal_factor_population * df[column].std())) + " -- " + str(round(df[column].mean()+removal_factor_population * df[column].std())))
    #outliers = df[np.abs(df[column] - df[column].mean()) > (3 * df[column].std())]
    outliers = df[df[column] < (df[column].mean() - removal_factor_population * df[column].std())]
    return outliers['userID'].unique()

# Identify speedsters: >3 SDs from population mean
outliers_reaction_times_instructions = set()
for column in ['timeConsentPage', 'timeInstruct1', 'timeInstruct2', 'timeAttentionCheck']:
    outliers_reaction_times_instructions.update(identify_outliers(reaction_times_instructions, column))

outliers_reaction_times_task = set()
for column in ['responseTime', 'explanationTime']:
    outliers_reaction_times_task.update(identify_outliers(reaction_times_task, column))
    
print("\nUserIDs of speedsters relative to all participants (instructions) :" + str(outliers_reaction_times_instructions))
print("UserIDs of speedsters relative to all participants (task) :" + str(outliers_reaction_times_task))

# Identify straightliners (repeatedly giving the same answer) from survey data
straightliners_survey = survey_data.groupby('userID').filter(lambda x: (x['responseNo'].var() == 0))
print("\nUserIDs of straightliners during survey: " + str(straightliners_survey['userID'].unique()))

# Identify straightliners (repeatedly giving the same answer) from task data
straightliners_task = task_performance.groupby('userID').filter(lambda x: (x['userPred'].var() == 0))
print("UserIDs of straightliners during task: " + str(straightliners_task['userID'].unique()))

# Identify attention failure in instruction (wrong answer to first question)
attention_failures_instruction = reaction_times_instructions[(reaction_times_instructions['responseAttentionCheck-pass2'] == 0)]['userID'].unique()
print("UserIDs of attention failures during instructions: " + str(attention_failures_instruction))

# Identify attention failure in survey (wrong answer to question X)
attention_failures_survey = survey_data[(survey_data['itemNo'] == 5) & (survey_data['checked'] != 1) & (survey_data['responseNo'] == 2)]['userID'].unique()
print("UserIDs of attention failures during survey: " + str(attention_failures_survey))

In [ ]:
# Combine all population outliers
all_outliers = outliers_reaction_times_instructions.union(outliers_reaction_times_task).union(straightliners_survey['userID'].unique()).union(straightliners_task['userID'].unique()).union(attention_failures_survey).union(attention_failures_instruction)

print("UserIDs of all outliers: " + str(all_outliers))
print("N: " + str(len(all_outliers)))

In [ ]:
# Remove population outliers from all datasets
reaction_times_instructions_cleaned = reaction_times_instructions[~reaction_times_instructions['userID'].isin(all_outliers)]
reaction_times_task_cleaned = reaction_times_task[~reaction_times_task['userID'].isin(all_outliers)]
task_performance_cleaned = task_performance[~task_performance['userID'].isin(all_outliers)]
survey_data_cleaned = survey_data[~survey_data['userID'].isin(all_outliers)]
demographics_cleaned = demographics[~demographics['userID'].isin(all_outliers)]

#### Wonky Materials?

Are there any that are systematically answered unusually quickly?
Here, we want to figure out if there are:

- any materials  >3 SDs from polulation mean
- any materials >3 SDs from person‘s own mean (to catch attention failures of individuals)

In [ ]:
# Identify wonky materials: RTs >3 SDs from person‘s own mean
# Define the factor with which std will be multiplied to find the admissible range
removal_factor_user_wise = 3
wonky_trial_user_percentage = 30

# Identify trials (=materials) where users performed more quickly than > 3SD from their own RT mean
response_time_outliers = reaction_times_task_cleaned[
    reaction_times_task_cleaned['responseTime'] < (
        reaction_times_task_cleaned.groupby('userID')['responseTime'].transform('mean') - 
        removal_factor_user_wise * reaction_times_task_cleaned.groupby('userID')['responseTime'].transform('std')
    )
]

explanation_time_outliers = reaction_times_task_cleaned[
    reaction_times_task_cleaned['explanationTime'] < (
        reaction_times_task_cleaned.groupby('userID')['explanationTime'].transform('mean') - 
        removal_factor_user_wise * reaction_times_task_cleaned.groupby('userID')['explanationTime'].transform('std')
    )
]

# Display the outliers for responseTime and explanationTime
print("\nUserIDs + trialNo of speedsters relative to own performance (response time):")
print(response_time_outliers[['userID', 'trialNo', 'responseTime']])

print("\nUserIDs + trialNo of speedsters relative to own performance (explanation time):")
print(explanation_time_outliers[['userID', 'trialNo', 'explanationTime']])

# Combine the outliers into a single DataFrame for analysis
outliers_combined = pd.concat([
    response_time_outliers[['userID', 'trialNo']],
    explanation_time_outliers[['userID', 'trialNo']]
]).drop_duplicates()

# Calculate the percentage of outliers for each trialNo(=material)
outlier_percentage = outliers_combined['trialNo'].value_counts(normalize=True) * 100

# Identify trialNo(=material) where at least 30% of participants were quicker than > 3 SDs of their own performance
wonky_trials = outlier_percentage[outlier_percentage >= wonky_trial_user_percentage].index.tolist()

print("\nList of trialNo where at least 30% of participants were quicker than > 3 SDs of their own performance:")
print(wonky_trials)

# Show specifics of wonky_trials (=material)
fields = ["trialNo", "truePred", "valQuality", "valProductivity", "valPunctuality", "valFeedback"]
wonky_trials_df = task_performance_cleaned[task_performance_cleaned['trialNo'].isin(wonky_trials)].groupby('trialNo').head(1)[fields]
print("\nSpecifics of wonky materials:")
print(wonky_trials_df)


# Filter out wonky trials from the dataframes - CAREFUL! LIKELY BREAKS BALANCE OF TRIALS!
#def remove_wonky_trials(df, wonky_trials):
#    return df[~df['trialNo'].isin(wonky_trials)]
# Remove wonky trials from all relevant dataframes
#reaction_times_task_cleaned = remove_wonky_trials(reaction_times_task_cleaned, wonky_trials)
#task_performance_cleaned = remove_wonky_trials(task_performance_cleaned, wonky_trials)

# 4. Eyeballing After Cleaning

In [ ]:
# Define function to calculate y-axis limit
def get_ylim(data, group_column, value_column):
    grouped = data.groupby(group_column)[value_column].agg(['mean', 'sem']).reset_index()
    max_ylim = (grouped['mean'] + 1.5*grouped['sem']).max()
    return 0, max_ylim

def get_material_ylim(data, group_column, material_column, value_column):
    grouped = data.groupby([group_column, material_column])[value_column].agg(['mean', 'sem']).reset_index()
    max_ylim = (grouped['mean'] + 1.2*grouped['sem']).max()
    return 0, max_ylim

# Initial plotting for eye-balling the data
plt.figure(figsize=(8, 4))

# Determine the maximum count for setting the y-limits for demographics
age_counts = demographics_cleaned.groupby(['age', 'cond']).size().max()
gender_counts = demographics_cleaned.groupby(['gender', 'cond']).size().max()
max_count_demo = max(age_counts, gender_counts)

# Plot Age distribution
plt.subplot(1, 2, 1)
sns.countplot(data=demographics_cleaned, x='age', hue='cond', palette=color_palette)
plt.title('Age Distribution by Condition')
plt.xlabel('Age')
plt.ylabel('Count')
plt.xticks(rotation=90)
plt.legend(loc='upper right')
plt.ylim(0, max_count_demo + 1)

# Plot Gender distribution
plt.subplot(1, 2, 2)
sns.countplot(data=demographics_cleaned, x='gender', hue='cond', palette=color_palette)
plt.title('Gender Distribution by Condition')
plt.xlabel('Gender')
plt.ylabel('Count')
plt.xticks(rotation=90)
plt.legend(loc='upper right')
plt.ylim(0, max_count_demo + 1)

plt.tight_layout()
plt.show()

# Look at Accuracy
# add a column 'orderedTrial' reflecting the temporal order of trials per participant
task_performance_cleaned = task_performance_cleaned.copy()
task_performance_cleaned['orderedTrial'] = task_performance_cleaned.groupby('userID').cumcount() + 1

plt.figure(figsize=(8, 4))
# Plot Mean Accuracy per user
# Compute mean accuracy and merge with demographics for plotting
mean_accuracy_per_user_cleaned = task_performance_cleaned.groupby('userID')['accuracy'].mean().reset_index()
mean_accuracy_per_user_cleaned = mean_accuracy_per_user_cleaned.merge(demographics_cleaned, on='userID')
sns.barplot(data=mean_accuracy_per_user_cleaned, x='cond', y='accuracy', palette=color_palette, errorbar='se')
plt.title('Mean Accuracy per User by Condition')
plt.xlabel('Condition')
plt.ylabel('Mean Accuracy')
plt.ylim(get_ylim(mean_accuracy_per_user_cleaned, 'cond', 'accuracy'))
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 4))
# Plot Mean Accuracy over users of each group over time
# (= trials chronologically ordered, to eyeball effects related to the duration of the study, fatigue, etc.)
mean_performance_per_trial_cleaned = task_performance_cleaned.groupby(['cond', 'orderedTrial']).agg({'accuracy': 'mean'}).reset_index()
sns.lineplot(data=mean_performance_per_trial_cleaned, x='orderedTrial', y='accuracy', hue='cond', marker='o', palette=color_palette)
plt.title('Mean Accuracy Over Users of Each Group on Each Trial - Chronologically Ordered')
plt.xlabel('Trial Number')
plt.ylabel('Mean Accuracy')
plt.legend(title='Condition')
plt.tight_layout()
plt.show()

plt.figure(figsize=(8, 4))
# Plot Mean Accuracy over users of each group on each material as bar plots
sns.barplot(data=task_performance_cleaned, x='trialNo', y='accuracy', hue='cond', palette=color_palette, errorbar='se')
plt.title('Mean Accuracy Over Users of Each Group on Each Material')
plt.xlabel('Trial Number')
plt.ylabel('Mean Accuracy')
plt.ylim(get_material_ylim(task_performance_cleaned, 'cond', 'trialNo', 'accuracy'))
plt.legend(title='Condition')
plt.tight_layout()
plt.show()

# Look at Reaction Times
# add a column 'orderedTrial' reflecting the temporal order of trials per participant
reaction_times_task_cleaned = reaction_times_task_cleaned.copy()
reaction_times_task_cleaned['orderedTrial'] = reaction_times_task_cleaned.groupby('userID').cumcount() + 1

max_ylim_times_mean_cleaned = max(get_ylim(reaction_times_task_cleaned, 'cond', 'responseTime'), get_ylim(reaction_times_task_cleaned, 'cond', 'explanationTime'))

plt.figure(figsize=(8, 4))
# Plot Reaction Times
plt.subplot(1, 2, 1)
sns.barplot(data=reaction_times_task_cleaned, x='cond', y='responseTime', palette=color_palette, errorbar='se')
plt.title('Mean Response Time by Condition')
plt.xlabel('Condition')
plt.ylabel('Response Time (ms)')
plt.ylim(max_ylim_times_mean_cleaned)

plt.subplot(1, 2, 2)
sns.barplot(data=reaction_times_task_cleaned, x='cond', y='explanationTime', palette=color_palette, errorbar='se')
plt.title('Mean Explanation Time by Condition')
plt.xlabel('Condition')
plt.ylabel('Explanation Time (ms)')
plt.ylim(max_ylim_times_mean_cleaned)
plt.tight_layout()
plt.show()

# Plot reaction times per user, divided into groups for more visibility
plt.figure(figsize=(8, 12))
groups_cleaned = reaction_times_task_cleaned['cond'].unique()

# Trials Chronologically Ordered, to Eyeball Effects Related to the Duration of the Study, Fatigue, etc.)
for i, group in enumerate(groups_cleaned, 1):
    plt.subplot(len(groups_cleaned), 1, i)
    subset_cleaned = reaction_times_task_cleaned[reaction_times_task_cleaned['cond'] == group]
    for user in subset_cleaned['userID'].unique():
        user_data_cleaned = subset_cleaned[subset_cleaned['userID'] == user]
        plt.plot(user_data_cleaned['orderedTrial'], user_data_cleaned['responseTime'], marker='o', linestyle='-', color='blue', alpha=0.6)
        plt.plot(user_data_cleaned['orderedTrial'], user_data_cleaned['explanationTime'], marker='x', linestyle='--', color='red', alpha=0.6)
    plt.title(f'Reaction Times for Group over Time: {group}')
    plt.xlabel('Trial Number (Chronological)')
    plt.ylabel('Time (ms)')
    plt.ylim(0, max(response_time_max, explanation_time_max))
    plt.legend(['Response Time', 'Explanation Time'], loc='upper right')
plt.tight_layout()
plt.show()

max_ylim_times_cleaned = max(get_material_ylim(reaction_times_task_cleaned, 'cond', 'trialNo', 'explanationTime'), get_material_ylim(reaction_times_task_cleaned, 'cond', 'trialNo', 'responseTime'))

# Plot Response Time per Material divided by group (to spot Effects Related to Materials)
plt.figure(figsize=(8, 4))
sns.barplot(x='trialNo', y='responseTime', hue='cond', data=reaction_times_task_cleaned, palette=color_palette, errorbar='se')
plt.title('Mean Response Time per Material Number')
plt.xlabel('Material Number')
plt.ylabel('Response Time (ms)')
plt.ylim(max_ylim_times_cleaned)
plt.legend(title='Condition')
plt.tight_layout()
plt.show()

# Plot Explanation Time per Material divided by group (to spot Effects Related to Materials)
plt.figure(figsize=(8, 4))
sns.barplot(x='trialNo', y='explanationTime', hue='cond', data=reaction_times_task_cleaned, palette=color_palette, errorbar='se')
plt.title('Mean Explanation Time per Material Number')
plt.xlabel('Material Number')
plt.ylabel('Explanation Time (ms)')
plt.ylim(max_ylim_times_cleaned)
plt.legend(title='Condition')
plt.tight_layout()
plt.show()

# 5. Descriptive Statistics
We'll compute descriptive statistics for the demographics and check if groups are significantly different in terms of age and gender.

## Average Time Needed

In [ ]:
mean_total_time_cleaned=np.mean(reaction_times_instructions['totalTime'])
print('Mean total time needed per participant:')
print(str(round(mean_total_time_cleaned/1000/60,2)) + ' minutes')

## Visualize Demographics (before and after cleaning)

In [ ]:
# Provide a summary of the number of participants in each group
participant_summary = demographics_cleaned.groupby('cond').size().reset_index(name='Number of Participants')

print("\nSummary of the number of participants in each group:")
print(participant_summary)

# make some plots:
# for uniform axes, get the frequency distribution of age / gender groups by condition
age_distribution_by_cond = demographics.groupby(['cond', 'age']).size().reset_index(name='count')
gender_distribution_by_cond = demographics.groupby(['cond', 'gender']).size().reset_index(name='count')

# Find the maximal frequency for demographics groups by condition
max_count_demo = max(max(age_distribution_by_cond.loc[age_distribution_by_cond.groupby('cond')['count'].idxmax()]['count']),max(gender_distribution_by_cond.loc[gender_distribution_by_cond.groupby('cond')['count'].idxmax()]['count']))

# Plot: Before data cleaning
plt.figure(figsize=(8, 4))

# Plot Age distribution
plt.subplot(1, 2, 1)
sns.countplot(data=demographics, x='age', hue='cond',palette=color_palette)
plt.title('Age Distribution by Condition Before Cleaning')
plt.xlabel('Age')
plt.ylabel('Count')
plt.xticks(rotation=90)
plt.legend(loc='upper right')
plt.ylim(0, max_count_demo+1)

# Plot Gender distribution
plt.subplot(1, 2, 2)
sns.countplot(data=demographics, x='gender', hue='cond',palette=color_palette)
plt.title('Gender Distribution by Condition Before Cleaning')
plt.xlabel('Gender')
plt.ylabel('Count')
plt.xticks(rotation=90)
plt.legend(loc='upper right')
plt.ylim(0, max_count_demo+1)

plt.tight_layout()
plt.show()

# Plot: After data cleaning
plt.figure(figsize=(8, 4))

# Plot Age distribution
plt.subplot(1, 2, 1)
sns.countplot(data=demographics_cleaned, x='age', hue='cond',palette=color_palette)
plt.title('Age Distribution by Condition After Cleaning')
plt.xlabel('Age')
plt.ylabel('Count')
plt.xticks(rotation=90)
plt.legend(loc='upper right')
plt.ylim(0, max_count_demo+1)

# Plot Gender distribution
plt.subplot(1, 2, 2)
sns.countplot(data=demographics_cleaned, x='gender', hue='cond',palette=color_palette)
plt.title('Gender Distribution by Condition After Cleaning')
plt.xlabel('Gender')
plt.ylabel('Count')
plt.xticks(rotation=90)
plt.legend(loc='upper right')
plt.ylim(0, max_count_demo+1)

plt.tight_layout()
plt.show()

## Test For Age Differences

In [ ]:
from scipy.stats import kruskal
import scikit_posthocs as sp

# Perform Kruskal-Wallis H Test for age distributions across conditions
groups = [demographics_cleaned[demographics_cleaned['cond'] == cond]['age'] for cond in demographics_cleaned['cond'].unique()]
kruskal_test_result = kruskal(*groups)

# Calculate Eta-Squared (η²)
H = kruskal_test_result.statistic
k = len(groups)
n = len(demographics)
eta_squared = (H - k + 1) / (n - 1)

print("Kruskal-Wallis H Test for Age Distributions:")
print(f"H-Statistic: {round(kruskal_test_result.statistic,3)}, p-value: {round(kruskal_test_result.pvalue,3)}")
print(f"Eta-Squared (η²) Effect Size: {abs(round(eta_squared,3))}")
print("\nNote about effect size Eta-squared η²: it indicates the proportion of the total variance of the dependent variable that can be explained by the independent variable. I.e., η² means that 20% of the variation observed in groups can be explained by age.")

# If the Kruskal-Wallis H test is significant, perform post hoc test: Dunn's Test
if kruskal_test_result.pvalue < 0.05:
    p_values_dunn_age = sp.posthoc_dunn(demographics_cleaned, val_col='age', group_col='cond', p_adjust='bonferroni')
    print("Post Hoc Comparisons using Dunn's Test with Bonferroni Correction:")
    print(p_values_dunn_age)

## Test For Gender Differences

In [ ]:
from scipy.stats import chi2_contingency, chi2, norm, shapiro, levene, kruskal
import itertools
import statsmodels.stats.multitest as smm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

# Chi-Square Test of Independence for Gender Distributions
contingency_table = pd.crosstab(demographics_cleaned['cond'], demographics_cleaned['gender'])
chi2_test_result = chi2_contingency(contingency_table)

print("Chi^2 Test of Independence for Gender Distributions:")
print(f"Chi^2 Statistic: {round(chi2_test_result[0],3)}, p-value: {round(chi2_test_result[1],3)}")
# if p > 0.05, H0 holds: we can assume that there is no dependence between age_group and condition

# Function to perform pairwise z-tests for proportions, for potential post-hoc comparisons
def pairwise_z_test(count1, nobs1, count2, nobs2):
    prop1 = count1 / nobs1
    prop2 = count2 / nobs2
    prop_pool = (count1 + count2) / (nobs1 + nobs2)
    se_pool = (prop_pool * (1 - prop_pool) * (1/nobs1 + 1/nobs2)) ** 0.5
    z_stat = (prop1 - prop2) / se_pool
    p_value = 2 * norm.cdf(-abs(z_stat))
    return p_value

# If the chi-square test is significant, perform post hoc tests
if chi2_test_result[1] < 0.05:
    # Generate all pairwise comparisons
    comparisons = list(itertools.combinations(contingency_table.index, 2))
    
    p_values = []
    for (group1, group2) in comparisons:
        for gender in contingency_table.columns:
            count1 = contingency_table.loc[group1, gender]
            count2 = contingency_table.loc[group2, gender]
            nobs1 = contingency_table.loc[group1].sum()
            nobs2 = contingency_table.loc[group2].sum()
            p_val = pairwise_z_test(count1, nobs1, count2, nobs2)
            p_values.append(p_val)
            print(f"Comparison {group1} vs {group2} for gender {gender}: p-value = {p_val}")
    
    # Adjust p-values for multiple comparisons using the Bonferroni correction
    reject, pvals_corrected, _, _ = smm.multipletests(p_values, alpha=0.05, method='bonferroni')
    
    # Display adjusted p-values
    for (group1, group2), p_val_corr in zip(comparisons, pvals_corrected):
        print(f"Adjusted p-value for {group1} vs {group2}: {p_val_corr}")

# 6. Statistical Analysis (finally!)
We'll analyze the accuracy of prediction and the mean feature importances for each group.

## First: Accuracy 

We will kick-off with the accuracy for group first. Initially, we planned to run a 2x2 ANOVA to compare mean performance of all groups. However, we have to keep in mind that ANOVA has two important assumptions: data needs to normally distributed and the variances need to be homogeneous. If these assumptions are not met, it is wiser to run the non-parametric alternative: Kruskal-Wallis H Test.

Here is the plan: 

1. Plot the accuracies again to remember the distributions.
2. Check Assumptions: Normality (Shapiro-Wilk test) and Homogeneity of Variances (Levene's test).
3. Perform ANOVA if assumptions are met.
4. If only normality is violated, perform ART ANOVA.
5. If both normality and homoscedasticity are violated, perform Kruskal-Wallis H Test.

### Step 1: Plot again to refresh our memory

In [ ]:
plt.figure(figsize=(8, 4))
# Plot Mean Accuracy per user
# Compute mean accuracy and merge with demographics for plotting
mean_accuracy_per_user_cleaned = task_performance_cleaned.groupby('userID')['accuracy'].mean().reset_index()
mean_accuracy_per_user_cleaned = mean_accuracy_per_user_cleaned.merge(demographics_cleaned, on='userID')
sns.barplot(data=mean_accuracy_per_user_cleaned, x='cond', y='accuracy', palette=color_palette, errorbar='se')
plt.title('Mean Accuracy per User by Condition')
plt.xlabel('Condition')
plt.ylabel('Mean Accuracy')
plt.ylim(get_ylim(mean_accuracy_per_user_cleaned, 'cond', 'accuracy'))
plt.tight_layout()
plt.show()

### Step 2: Check Assumptions

In [ ]:
# Check Normality using Shapiro-Wilk test (if p<0.05, assume data is NOT normal)
normality_results = task_performance_cleaned.groupby('cond')['accuracy'].apply(shapiro)
normality_passed = all(p.pvalue > 0.05 for p in normality_results)
print("Shapiro-Wilk Test for Normality:")
for cond, result in normality_results.items():
    print(f"Condition {cond}: W-statistic = {round(result.statistic,3)}, p-value = {round(result.pvalue,3)}")

# Check Homogeneity of Variances using Levene's test (if p<0.05, assume data is NOT homoscedastic)
levene_stat, levene_pvalue = levene(*[task_performance_cleaned[task_performance_cleaned['cond'] == cond]['accuracy'] for cond in task_performance_cleaned['cond'].unique()])
homogeneity_passed = levene_pvalue > 0.05
print(f"\nLevene's Test for Homogeneity of Variances: Statistic = {round(levene_stat,3)}, p-value = {round(levene_pvalue,3)}")

### Step 3-5: Perform ANOVA if assumptions are met, ART ANOVA if partially met, & Kruskal-Wallis H Test otherwise

In [ ]:
from rpy2 import robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects.packages import importr


# Generate "referent" and "exp" columns using assign to avoid SettingWithCopyWarning
task_performance_cleaned = task_performance_cleaned.assign(
    referent=task_performance_cleaned['cond'].str[0],
    exp=task_performance_cleaned['cond'].str.split('-').str[1]
)

# Convert "referent" and "exp" columns to categorical type in Python
task_performance_cleaned['referent'] = task_performance_cleaned['referent'].astype('category')
task_performance_cleaned['exp'] = task_performance_cleaned['exp'].astype('category')

print("\nVisualize results")
plt.figure(figsize=(8, 4))
sns.pointplot(data=task_performance_cleaned, x='referent', y='accuracy', hue='exp', palette=color_palette_interaction, dodge=True, markers=['o', 's'], capsize=0.1, errwidth=1, errorbar='se')
plt.title('Interaction Plot: Mean Accuracy by Referent and Exp')
plt.xlabel('Referent')
plt.ylabel('Mean Accuracy')
plt.legend(title='Exp')
plt.tight_layout()
plt.show()

if normality_passed and homogeneity_passed:
    # Perform 2x2 ANOVA
    model = ols('accuracy ~ C(referent) * C(exp)', data=task_performance_cleaned).fit()
    anova_results = anova_lm(model)
    print("ANOVA Results:")
    print(round(anova_results,3))
    print("\nANOVA results table explained:")
    print("1. row: results for main effect of factor 'referent'")
    print("2. row: results for main effect of factor 'exp'")
    print("3. row: results for the interaction of both factors. If this is singificant, the effect of one factor depends on the value of the other factor.")
    print("4. row: residual, the variation in the dependent variable (accuracy) that isn't explained by the independent variables (referent or factor)")
    # Perform post hoc test if interaction of ANOVA is significant
    if anova_results["PR(>F)"][2] < 0.05:
        posthoc_result = sp.posthoc_dunn(task_performance_cleaned, val_col='accuracy', group_col='cond', p_adjust='bonferroni')
        print("\nPost Hoc Comparisons (Interaction) using Dunn's Test:")
        print(round(posthoc_result,3))
        
else:
    # Perform Kruskal-Wallis H Test for Accuracy
    groups = [task_performance_cleaned[task_performance_cleaned['cond'] == cond]['accuracy'] for cond in task_performance_cleaned['cond'].unique()]
    kruskal_test_result = kruskal(*groups)

    # Calculate Eta-Squared (η²)
    H = kruskal_test_result.statistic
    k = len(groups)
    n = len(task_performance_cleaned)
    eta_squared = (H - k + 1) / (n - 1)

    print("Kruskal-Wallis H Test for Accuracy:")
    print(f"Statistic: {round(kruskal_test_result.statistic,3)}, p-value: {round(kruskal_test_result.pvalue,3)}")
    print(f"Eta-Squared (η²) Effect Size: {round(eta_squared,3)}")

    # Perform Dunn's test for post hoc comparisons if Kruskal-Wallis test is significant
    if kruskal_test_result.pvalue < 0.05:
        posthoc_result = sp.posthoc_dunn(task_performance_cleaned, val_col='accuracy', group_col='cond', p_adjust='bonferroni')
        print("Post Hoc Comparisons using Dunn's Test with Bonferroni Correction:")
        print(round(posthoc_result,3))

## Second: User Judgements from Survey

Let's turn to the survey information. Two measures will be considered:

A. Distance between ground truth importance and user evaluation

B. Analysis of Survey Responses

### A. Compute and Evaluate Distance Measure Between Ground Truth Importance And User Judgements

In [ ]:
# Filter to keep only rows where itemNo == 1 and checked == 1
survey_data_importance = survey_data_cleaned[(survey_data_cleaned['itemNo'] == 1) & (survey_data_cleaned['checked'] == 1)]

# Drop duplicate rows based on userID, keeping only the first occurrence
survey_data_importance = survey_data_importance.drop_duplicates(subset='userID')

# Define a function to transform the values into "high", "medium", and "low" based on quantiles
def transform_quantile_ranking(row):
    scores = [
        row['importanceQuality'],
        row['importanceProductivity'],
        row['importancePunctuality'],
        row['importanceFeedback']
    ]
    high_threshold = pd.Series(scores).quantile(0.66)
    low_threshold = pd.Series(scores).quantile(0.33)
    return [
        'high' if score > high_threshold else 'medium' if score > low_threshold else 'low' 
        for score in scores
    ]

# Apply the transformation to the relevant columns using assign
rankings = survey_data_importance.apply(transform_quantile_ranking, axis=1)
rankings_df = pd.DataFrame(rankings.tolist(), columns=['Quality', 'Productivity', 'Punctuality', 'Feedback'])

survey_data_importance = pd.concat([survey_data_importance.reset_index(drop=True), rankings_df], axis=1)

# Compute the distance measure based on the specified criteria
def compute_distance_measure(row):
    score = 0
    if row['Quality'] == 'high':
        score += 1
    if row['Productivity'] == 'high':
        score += 1
    if row['Punctuality'] == 'medium':
        score += 1
    if row['Feedback'] == 'low':
        score += 1
    return score

# Apply the function to compute the distance measure
survey_data_importance['distance_measure'] = survey_data_importance.apply(compute_distance_measure, axis=1)

# Visualize mean per group for the distance_measure
plt.figure(figsize=(8, 4))
# Plot Mean Distance Measure per User
sns.barplot(data=survey_data_importance, x='cond', y='distance_measure', palette=color_palette, errorbar='se')
plt.title('Mean Distance to Ground Truth by Condition (max=4, min=0)')
plt.xlabel('Condition')
plt.ylabel('Distance Measure')
plt.tight_layout()
plt.show()

# generate factor columns
survey_data_importance['referent'] = survey_data_importance['cond'].str[0]  # 's' or 'o'
survey_data_importance['exp'] = survey_data_importance['cond'].str.split('-').str[1]  # 'cfe' or 'con'
# Convert "referent" and "exp" columns to categorical type in Python
survey_data_importance['referent'] = survey_data_importance['referent'].astype('category')
survey_data_importance['exp'] = survey_data_importance['exp'].astype('category')

# Check assumption 1) Normality (if Shapiro-Wilk p<0.05, assume data is NOT normal)
normality_results = survey_data_importance.groupby('cond')['distance_measure'].apply(shapiro)
normality_passed = all(p.pvalue > 0.05 for p in normality_results)
print("Shapiro-Wilk Test for Normality:")
for cond, result in normality_results.items():
    print(f"Condition {cond}: W-statistic = {round(result.statistic,3)}, p-value = {round(result.pvalue,3)}")

# Check Homogeneity of Variances using Levene's test (if p<0.05, assume data is NOT homoscedastic)
levene_stat, levene_pvalue = levene(*[survey_data_importance[survey_data_importance['cond'] == cond]['distance_measure'] for cond in survey_data_importance['cond'].unique()])
homogeneity_passed = levene_pvalue > 0.05
print(f"\nLevene's Test for Homogeneity of Variances: Statistic = {round(levene_stat,3)}, p-value = {round(levene_pvalue,3)}")

print("\nVisualize results")
plt.figure(figsize=(8, 4))
sns.pointplot(data=survey_data_importance, x='referent', y='distance_measure', hue='exp', palette=color_palette_interaction, dodge=True, markers=['o', 's'], capsize=0.1, errwidth=1, errorbar='se')
plt.title('Interaction Plot: Mean Distance Measure by Referent and Exp')
plt.xlabel('Referent')
plt.ylabel('Mean Distance from Ground Truth')
plt.legend(title='Exp')
plt.tight_layout()
plt.show()

if normality_passed and homogeneity_passed:
    # Perform 2x2 ANOVA
    model = ols('distance_measure ~ C(referent) * C(exp)', data=survey_data_importance).fit()
    anova_results = anova_lm(model)
    print("ANOVA Results:")
    print(round(anova_results,3))
    print("\nANOVA results table explained:")
    print("1. row: results for main effect of factor 'referent'")
    print("2. row: results for main effect of factor 'exp'")
    print("3. row: results for the interaction of both factors. If this is singificant, the effect of one factor depends on the value of the other factor.")
    print("4. row: residual, the variation in the dependent variable (accuracy) that isn't explained by the independent variables (referent or factor)")
    
    # Perform post hoc test if interaction of ANOVA is significant
    if anova_results["PR(>F)"][2] < 0.05:
        posthoc_result = sp.posthoc_dunn(survey_data_importance, val_col='distance_measure', group_col='cond', p_adjust='bonferroni')
        print("\nPost Hoc Comparisons (Interaction) using Dunn's Test:")
        print(round(posthoc_result,3))
        
else:
    # Perform Kruskal-Wallis H Test for Accuracy
    groups = [survey_data_importance[survey_data_importance['cond'] == cond]['distance_measure'] for cond in survey_data_importance['cond'].unique()]
    kruskal_test_result = kruskal(*groups)

    # Calculate Eta-Squared (η²)
    H = kruskal_test_result.statistic
    k = len(groups)
    n = len(task_performance_cleaned)
    eta_squared = (H - k + 1) / (n - 1)

    print("Kruskal-Wallis H Test for Distance Masure:")
    print(f"Statistic: {round(kruskal_test_result.statistic,3)}, p-value: {round(kruskal_test_result.pvalue,3)}")
    print(f"Eta-Squared (η²) Effect Size: {round(eta_squared,3)}")

    # Perform Dunn's test for post hoc comparisons if Kruskal-Wallis test is significant
    if kruskal_test_result.pvalue < 0.05:
        posthoc_result = sp.posthoc_dunn(survey_data_importancetask_performance_cleaned, val_col='distance_measure', group_col='cond', p_adjust='bonferroni')
        print("Post Hoc Comparisons using Dunn's Test with Bonferroni Correction:")
        print(round(posthoc_result,3))

### B. Analyze Survey Responses Regarding Satisfaction and Trust

#### Satisfaction

In [ ]:
# Filter only rows where checked == 1
survey_data_checked = survey_data_cleaned[survey_data_cleaned['checked'] == 1]

# Define the items for Explanation Satisfaction Scale and Explanation Trust Scale
satisfaction_items = [0, 1, 2, 3, 4, 6, 7, 8] #[1, 2, 3, 4, 5, 7, 8, 9] 
trust_items = [9, 10, 11, 12, 13, 14, 15, 16]

# Define labels for the plots
satisfaction_labels = {
    0: 'Understanding', 1: 'Satisfaction (reverse scored)', 2: 'Sufficient detail', 3: 'Completeness',
    4: 'How to use', 6: 'Usefulness (reverse scored)', 7: 'Accuracy', 8: 'Appropriate trust'
}

trust_labels = {
    9: 'Confidence (reverse scored)', 10: 'Predictability', 11: 'Reliance', 12: 'Fair Evaluation (reverse scored)',
    13: 'Efficiency', 14: 'Wariness (reverse scored)', 15: 'Improvement over human performance', 16: 'Liking of system'} ##, 17: 'Liking of system}

# Plot average judgments per group for Explanation Satisfaction Scale
plt.figure(figsize=(16, 10))
for i, item in enumerate(satisfaction_items, 1):
    plt.subplot(2, 4, i)
    sns.barplot(data=survey_data_checked[survey_data_checked['itemNo'] == item], x='cond', y='responseNo', palette=color_palette, errorbar='se')
    plt.title(satisfaction_labels[item])
    plt.xlabel('Condition')
    plt.ylabel('Mean Judgment')
plt.suptitle('Mean Judgments for Explanation Satisfaction Scale')
plt.tight_layout()
plt.show()

# Recode reverse scored items
survey_data_checked = survey_data_checked.assign(
    response_recode=survey_data_checked['responseNo']
)

# To get a Correct Summed Score, reverse code relevant items
survey_data_checked.loc[survey_data_checked['itemNo'].isin([1, 6, 9, 12, 14]), 'response_recode'] = 6 - survey_data_checked['responseNo']

# Compute summed scores for satisfaction and trust scales
satisfaction_scores = survey_data_checked[survey_data_checked['itemNo'].isin(satisfaction_items)].groupby('userID')['response_recode'].sum()
trust_scores = survey_data_checked[survey_data_checked['itemNo'].isin(trust_items)].groupby('userID')['response_recode'].sum()

# Add summed scores to the original DataFrame
survey_data_checked = survey_data_checked.merge(satisfaction_scores.rename('satisfaction_score'), on='userID')
survey_data_checked = survey_data_checked.merge(trust_scores.rename('trust_score'), on='userID')

# Plot mean satisfaction scores per group
plt.figure(figsize=(8, 4))
sns.barplot(data=survey_data_checked, x='cond', y='satisfaction_score', palette=color_palette, errorbar='se')
#sns.boxplot(data=survey_data_checked, x='cond', y='satisfaction_score',palette=color_palette)
plt.title('Mean Satisfaction Scores by Condition (max=40)')
plt.xlabel('Condition')
plt.ylabel('Satisfaction Score')
plt.show()

# Perform statistics following the now familiar logic: test for ANOVA assumptions, continue accordingly
# generate factor columns
survey_data_checked['referent'] = survey_data_checked['cond'].str[0]  # 's' or 'o'
survey_data_checked['exp'] = survey_data_checked['cond'].str.split('-').str[1]  # 'cfe' or 'con'
# Convert "referent" and "exp" columns to categorical type in Python
survey_data_checked['referent'] = survey_data_checked['referent'].astype('category')
survey_data_checked['exp'] = survey_data_checked['exp'].astype('category')

### statistical analysis of satisfaction:
# Check assumption 1) Normality (if Shapiro-Wilk p<0.05, assume data is NOT normal)
normality_results = survey_data_checked.groupby('cond')['satisfaction_score'].apply(shapiro)
normality_passed = all(p.pvalue > 0.05 for p in normality_results)
print("Shapiro-Wilk Test for Normality:")
for cond, result in normality_results.items():
    print(f"Condition {cond}: W-statistic = {round(result.statistic,3)}, p-value = {round(result.pvalue,3)}")

# Check Homogeneity of Variances using Levene's test (if p<0.05, assume data is NOT homoscedastic)
levene_stat, levene_pvalue = levene(*[survey_data_checked[survey_data_checked['cond'] == cond]['satisfaction_score'] for cond in survey_data_checked['cond'].unique()])
homogeneity_passed = levene_pvalue > 0.05
print(f"\nLevene's Test for Homogeneity of Variances: Statistic = {round(levene_stat,3)}, p-value = {round(levene_pvalue,3)}")

print("\nVisualize results")
plt.figure(figsize=(8, 4))
sns.pointplot(data=survey_data_checked, x='referent', y='satisfaction_score', hue='exp', palette=color_palette_interaction, dodge=True, markers=['o', 's'], capsize=0.1, errwidth=1, errorbar='se')
plt.title('Interaction Plot: Mean Satisfaction Score by Referent and Exp')
plt.xlabel('Referent')
plt.ylabel('Mean Satisfaction Score')
plt.legend(title='Exp')
plt.tight_layout()
plt.show()

if normality_passed and homogeneity_passed:
    # Perform 2x2 ANOVA
    model = ols('satisfaction_score ~ C(referent) * C(exp)', data=survey_data_checked).fit()
    anova_results = anova_lm(model)
    print("ANOVA Results:")
    print(round(anova_results,3))
    print("\nANOVA results table explained:")
    print("1. row: results for main effect of factor 'referent'")
    print("2. row: results for main effect of factor 'exp'")
    print("3. row: results for the interaction of both factors. If this is singificant, the effect of one factor depends on the value of the other factor.")
    print("4. row: residual, the variation in the dependent variable (accuracy) that isn't explained by the independent variables (referent or factor)")
    
    # Perform post hoc test if interaction of ANOVA is significant
    if anova_results["PR(>F)"][2] < 0.05:
        posthoc_result = sp.posthoc_dunn(survey_data_checked, val_col='satisfaction_score', group_col='cond', p_adjust='bonferroni')
        print("\nPost Hoc Comparisons (Interaction) using Dunn's Test:")
        print(round(posthoc_result,3))
        
else:
    # Perform Kruskal-Wallis H Test for Accuracy
    groups = [survey_data_checked[survey_data_checked['cond'] == cond]['satisfaction_score'] for cond in survey_data_checked['cond'].unique()]
    kruskal_test_result = kruskal(*groups)

    # Calculate Eta-Squared (η²)
    H = kruskal_test_result.statistic
    k = len(groups)
    n = len(task_performance_cleaned)
    eta_squared = (H - k + 1) / (n - 1)

    print("\nKruskal-Wallis H Test for Satisfaction Score:")
    print(f"Statistic: {round(kruskal_test_result.statistic,3)}, p-value: {round(kruskal_test_result.pvalue,3)}")
    print(f"Eta-Squared (η²) Effect Size: {round(eta_squared,3)}")

    # Perform Dunn's test for post hoc comparisons if Kruskal-Wallis test is significant
    if kruskal_test_result.pvalue < 0.05:
        posthoc_result = sp.posthoc_dunn(survey_data_checked, val_col='satisfaction_score', group_col='cond', p_adjust='bonferroni')
        print("Post Hoc Comparisons using Dunn's Test with Bonferroni Correction:")
        print(round(posthoc_result,3))  

#### Trust

In [ ]:
# Plot average judgments per group for Explanation Trust Scale
plt.figure(figsize=(16, 10))
for i, item in enumerate(trust_items, 1):
    plt.subplot(2, 4, i)
    sns.barplot(data=survey_data_checked[survey_data_checked['itemNo'] == item], x='cond', y='responseNo', palette=color_palette, errorbar='se')
    plt.title(trust_labels[item])
    plt.xlabel('Condition')
    plt.ylabel('Mean Judgment')
plt.suptitle('Mean Judgments for Explanation Trust Scale')
plt.tight_layout()
plt.show()

# Plot mean trust scores per group
plt.figure(figsize=(8, 4))
sns.barplot(data=survey_data_checked, x='cond', y='trust_score', palette=color_palette, errorbar='se')
plt.title('Mean Trust Scores by Condition')
plt.xlabel('Condition')
plt.ylabel('Trust Score')
plt.show()

### statistical analysis of trust:
# Check assumption 1) Normality (if Shapiro-Wilk p<0.05, assume data is NOT normal)
normality_results = survey_data_checked.groupby('cond')['trust_score'].apply(shapiro)
normality_passed = all(p.pvalue > 0.05 for p in normality_results)
print("Shapiro-Wilk Test for Normality:")
for cond, result in normality_results.items():
    print(f"Condition {cond}: W-statistic = {round(result.statistic,3)}, p-value = {round(result.pvalue,3)}")

# Check Homogeneity of Variances using Levene's test (if p<0.05, assume data is NOT homoscedastic)
levene_stat, levene_pvalue = levene(*[survey_data_checked[survey_data_checked['cond'] == cond]['trust_score'] for cond in survey_data_checked['cond'].unique()])
homogeneity_passed = levene_pvalue > 0.05
print(f"\nLevene's Test for Homogeneity of Variances: Statistic = {round(levene_stat,3)}, p-value = {round(levene_pvalue,3)}")

print("\nVisualize results")
plt.figure(figsize=(8, 4))
sns.pointplot(data=survey_data_checked, x='referent', y='trust_score', hue='exp', palette=color_palette_interaction, dodge=True, markers=['o', 's'], capsize=0.1, errwidth=1, errorbar='se')
plt.title('Interaction Plot: Mean Trust Score by Referent and Exp')
plt.xlabel('Referent')
plt.ylabel('Mean Trust Score')
plt.legend(title='Exp')
plt.tight_layout()
plt.show()

if normality_passed and homogeneity_passed:
    # Perform 2x2 ANOVA
    model = ols('trust_score ~ C(referent) * C(exp)', data=survey_data_checked).fit()
    anova_results = anova_lm(model)
    print("ANOVA Results:")
    print(round(anova_results,3))
    print("\nANOVA results table explained:")
    print("1. row: results for main effect of factor 'referent'")
    print("2. row: results for main effect of factor 'exp'")
    print("3. row: results for the interaction of both factors. If this is singificant, the effect of one factor depends on the value of the other factor.")
    print("4. row: residual, the variation in the dependent variable (accuracy) that isn't explained by the independent variables (referent or factor)")
    
    # Perform post hoc test if interaction of ANOVA is significant
    if anova_results["PR(>F)"][2] < 0.05:
        posthoc_result = sp.posthoc_dunn(survey_data_checked, val_col='trust_score', group_col='cond', p_adjust='bonferroni')
        print("\nPost Hoc Comparisons (Interaction) using Dunn's Test:")
        print(round(posthoc_result,3))
        
elif not normality_passed and homogeneity_passed:
    # Suppress R-specific output to avoid confusions:
    with contextlib.redirect_stdout(open(os.devnull, 'w')), contextlib.redirect_stderr(open(os.devnull, 'w')):
        # Enable the conversion between pandas DataFrames and R data frames
        pandas2ri.activate()
        # Convert columns to factors and prepare data for R
        survey_data_checked_r = pandas2ri.py2rpy(survey_data_checked)
        survey_data_checked_r = robjects.r['data.frame'](survey_data_checked_r)
        robjects.r.assign("survey_data_importance_r", survey_data_checked_r)
        
        robjects.r('''
            survey_data_checked_r$referent <- as.factor(survey_data_checked_r$referent)
            survey_data_checked_r$exp <- as.factor(survey_data_checked_r$exp)
        ''')
        
        # Perform ART ANOVA using ARTool in R
        art = importr('ARTool')
        survey_data_checked_r = pandas2ri.py2rpy(survey_data_checked)
        
        formula = robjects.Formula('trust_score ~ referent * exp')
        environment = formula.environment
        environment['data'] = survey_data_checked_r
        
        # Perform the ART ANOVA
        art_model = art.art(formula, data=survey_data_checked_r)
        anova_results = art.anova_art(art_model,verbose=True)
        
    print("Aligned Rank Transform (ART) ANOVA Results:")
    print(anova_results)
    print("\nANOVA results table explained:")
    print("1. row: results for main effect of factor 'referent'")
    print("2. row: results for main effect of factor 'exp'")
    print("3. row: results for the interaction of both factors. If this is singificant, the effect of one factor depends on the value of the other factor.")
    print("\nVisualize results")
    plt.figure(figsize=(8, 4))
    sns.pointplot(data=survey_data_checked, x='referent', y='trust_score', hue='exp', palette=color_palette_interaction, dodge=True, markers=['o', 's'], capsize=0.1, errwidth=1, errorbar='se')
    plt.title('Interaction Plot: Mean Trust Score by Referent and Exp')
    plt.xlabel('Referent')
    plt.ylabel('Mean Trust Score')
    plt.legend(title='Exp')
    plt.tight_layout()
    plt.show()
    # Perform post hoc test if interaction of ANOVA is significant
    if anova_results.rx2('Pr(>F)')[2] < 0.05:
        posthoc_result = sp.posthoc_dunn(survey_data_checked, val_col='trust_score', group_col='cond', p_adjust='bonferroni')
        print("\nPost Hoc Comparisons (Interaction) using Dunn's Test:")
        print(round(posthoc_result,3))
        
else:
    # Perform Kruskal-Wallis H Test for Accuracy
    groups = [survey_data_checked[survey_data_checked['cond'] == cond]['trust_score'] for cond in survey_data_checked['cond'].unique()]
    kruskal_test_result = kruskal(*groups)

    # Calculate Eta-Squared (η²)
    H = kruskal_test_result.statistic
    k = len(groups)
    n = len(task_performance_cleaned)
    eta_squared = (H - k + 1) / (n - 1)

    print("\nKruskal-Wallis H Test for the Trust Score:")
    print(f"Statistic: {round(kruskal_test_result.statistic,3)}, p-value: {round(kruskal_test_result.pvalue,3)}")
    print(f"Eta-Squared (η²) Effect Size: {round(eta_squared,3)}")

    # Perform Dunn's test for post hoc comparisons if Kruskal-Wallis test is significant
    if kruskal_test_result.pvalue < 0.05:
        posthoc_result = sp.posthoc_dunn(survey_data_checked, val_col='trust_score', group_col='cond', p_adjust='bonferroni')
        print("Post Hoc Comparisons using Dunn's Test with Bonferroni Correction:")
        print(round(posthoc_result,3))  
